In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import random

def handle_no_such_element_exception(data_extraction_task):
    try:
        return data_extraction_task()
    except NoSuchElementException as e:
        return None

# Create a Chrome web driver instance
driver = webdriver.Chrome(service=Service())

# Connect to the target page
url = "https://www.booking.com/searchresults.de.html?label=gen173nr-1BCAEoggI46AdIM1gEaDuIAQGYAQe4ARfIAQzYAQHoAQGIAgGoAgO4Ap-wuLoGwAIB0gIkMWVhOGIyYzUtNzkwZC00ODI3LWE5Y2QtNzY3Mjg5YjA0NGI12AIF4AIB&sid=4daa01aeadda31e7f02b8ddc59e9f35f&aid=304142&ss=Berlin&ssne=Berlin&ssne_untouched=Berlin&efdco=1&lang=de&sb=1&src_elem=sb&dest_id=-1746443&dest_type=city&checkin=2025-05-05&checkout=2025-05-10&ltfd=5%3A1%3A6-2025%3A1%3A&group_adults=2&no_rooms=1&group_children=0&order=distance_from_search"
driver.get(url)

# Handle the sign-in alert
try:
    close_button = WebDriverWait(driver, 12).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[role=\"dialog\"] button[aria-label=\"Dismiss sign-in info.\"]"))
    )
    close_button.click()
except Exception as e:
    print("Sign-in modal did not appear, continuing...")

# Scroll down the page in a human-like fashion with a maximum scroll time
scroll_pause_time = 3  # Pause time between scrolls
max_scroll_time = 320  # Maximum scroll time in seconds
start_time = time.time()
last_height = driver.execute_script("return document.body.scrollHeight")
load_more_button_found = False
max_button_clicks = 30  # Maximum number of button clicks

while True:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page
    time.sleep(scroll_pause_time)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height or (time.time() - start_time) > max_scroll_time:
        break
    last_height = new_height

    # Check if "Load More Results" button is present and click it
    if max_button_clicks > 0:
        try:
            load_more_button = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "button.a83ed08757.c21c56c305.bf0537ecb5.f671049264.af7297d90d.c0e0affd09")
                )
            )
            load_more_button.click()
            load_more_button_found = True
            print("Clicked 'Load More Results' button")
            # Add a small pause after clicking the button
            time.sleep(2)
            max_button_clicks -= 1
        except TimeoutException:
            pass  # Button not yet available, continue scrolling

# If "Load More Results" button was found, wait an additional time for final loading
if load_more_button_found:
    time.sleep(5)  # Adjust this wait time as needed

# Where to store the scraped data
items = []


# Select all property items on the page
property_items = driver.find_elements(By.CSS_SELECTOR, "[data-testid=\"property-card\"]")

print(f"Found {len(property_items)} property items.")


Sign-in modal did not appear, continuing...
Clicked 'Load More Results' button
Clicked 'Load More Results' button
Clicked 'Load More Results' button
Clicked 'Load More Results' button
Clicked 'Load More Results' button
Clicked 'Load More Results' button
Clicked 'Load More Results' button
Clicked 'Load More Results' button
Clicked 'Load More Results' button
Clicked 'Load More Results' button
Clicked 'Load More Results' button
Clicked 'Load More Results' button
Clicked 'Load More Results' button
Clicked 'Load More Results' button
Clicked 'Load More Results' button
Clicked 'Load More Results' button
Clicked 'Load More Results' button
Clicked 'Load More Results' button
Clicked 'Load More Results' button
Clicked 'Load More Results' button
Clicked 'Load More Results' button
Clicked 'Load More Results' button
Clicked 'Load More Results' button
Found 646 property items.


In [10]:

import csv

for property_item in property_items:
    # scraping logic...
    url = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "a[data-testid=\"property-card-desktop-single-image\"]").get_attribute("href"))
    image = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "img[data-testid=\"image\"]").get_attribute("src"))
    time.sleep(random.uniform(1, 2))
    title = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "[data-testid=\"title\"]").text)
    address = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "[data-testid=\"address\"]").text)
    distance = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "[data-testid=\"distance\"]").text)

    review_score = None
    review_count = None
    review_text = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "[data-testid=\"review-score\"]").text)
    if review_text is not None:
    # split the review string by \n
        parts = review_text.split("\n")

    # process each part
    for part in parts:
        part = part.strip()
        # check if this part is a number (potential review score)
        if part.replace(".", "", 1).isdigit():
            review_score = float(part)
        # check if it contains the "reviews" string
        elif "reviews" in part:
            # extract the number before "reviews"
            review_count = int(part.split(" ")[0].replace(",", ""))
    decription = handle_no_such_element_exception(lambda: property_item.find_element(By.CSS_SELECTOR, "[data-testid=\"recommended-units\"]").text)
    time.sleep(random.uniform(1, 2))
    price_element = handle_no_such_element_exception(lambda: (property_item.find_element(By.CSS_SELECTOR, "[data-testid=\"availability-rate-information\"]")))
    if price_element is not None:
        original_price = handle_no_such_element_exception(lambda: (
            price_element.find_element(By.CSS_SELECTOR, "[aria-hidden=\"true\"]:not([data-testid])").text.replace(",", "")
        ))
        price = handle_no_such_element_exception(lambda: (
            price_element.find_element(By.CSS_SELECTOR, "[data-testid=\"price-and-discounted-price\"]").text.replace(",", "")
        ))

    # populate a new item with the scraped data
    item = {
    "url": url,
    "image": image,
    "title": title,
    "address": address,
    "distance": distance,
    "review_score": review_score,
    "review_count": review_count,
    "decription": decription,
    "original_price": original_price,
    "price": price
    }
    # add the new item to the list of scraped items
    items.append(item)

output_file = "properties.csv"

with open(output_file, mode="w", newline="", encoding="utf-8") as file:
        #create a CSV writer object
    writer = csv.DictWriter(file, fieldnames=["url", "image", "title", "address", "distance", "review_score", "review_count", "decription", "original_price", "price"])

    # write the header row
    writer.writeheader()

    # write each item as a row in the CSV
    writer.writerows(items)
    driver.quit()


In [19]:
import re


hotel_url_names = []

for item in items:
    url = item['url']
    match = re.search(r'hotel/de/(.+?)\.de\.html', url)
    if match:
        hotel_url_name = match.group(1)
        hotel_url_names.append(hotel_url_name)

print(hotel_url_names)

['adlon-kempinski-berlin', 'nexliving-berlin-brandenburger-tor', 'marriott-am-potsdamer-platz', 'chateau-royal-gmbh-berlin62', 'hostel-near-the-brandenburg-gate', 'the-ritz-carlton-berlin', 'fairytale-in-sony-center-berlin', 'westingrand', 'the-mandala', 'maritimberlin', 'albrechtshof', 'hotelallegra', 'jollyvivaldi', 'leonardo-berlin-mitte', 'eurostars-berlin', 'clipper-city-home', 'best-western-berlin-mitte', 'grand-hyatt-berlin', 'melia-berlin', 'gendarmenmarkt', 'the-mandala-berlin', 'nhberlinmitte', 'steigenberger-am-kanzleramt', 'artist-riverside-spa', 'meininger-berlin-hauptbahnhof', 'intercityhotel-berlin-hbf', 'gat-point-charlie', 'rocco-forte-de-rome', 'titanic-gendarmenmarkt-berlin', 'hilton-berlin', 'precise-tale-berlin', 'wilde-aparthotels-by-staycity-berlin-checkpoint-charlie', 'winters-berlin-mitte-the-wall-checkpoint-charlie', 'dietrich-bonhoeffer', 'baxpax-downtown-hostel', 'ibis-berlin-hauptbahnhof', 'motel-one-berlin-hauptbahnhof', 'luxury-apartment-berlin-mitte-berl

In [20]:
# ADD keys
import pandas as pd
df = pd.DataFrame(items)

df["Hotel_key"] = (hotel_url_names)
print(df)
df.to_json('properties.json', orient='records', lines=True)


                                                   url  \
0    https://www.booking.com/hotel/de/adlon-kempins...   
1    https://www.booking.com/hotel/de/nexliving-ber...   
2    https://www.booking.com/hotel/de/marriott-am-p...   
3    https://www.booking.com/hotel/de/chateau-royal...   
4    https://www.booking.com/hotel/de/hostel-near-t...   
..                                                 ...   
641  https://www.booking.com/hotel/de/down-town-mot...   
642  https://www.booking.com/hotel/de/ferienwohnung...   
643  https://www.booking.com/hotel/de/neu-helgoland...   
644  https://www.booking.com/hotel/de/dameritzseeho...   
645  https://www.booking.com/hotel/de/ferienhaus-am...   

                                                 image  \
0    https://cf.bstatic.com/xdata/images/hotel/squa...   
1    https://cf.bstatic.com/xdata/images/hotel/squa...   
2    https://cf.bstatic.com/xdata/images/hotel/squa...   
3    https://cf.bstatic.com/xdata/images/hotel/squa...   
4    https://

In [ ]:
for hotel_url_name in hotel_url_names:
    !python run.py {hotel_url_name} "de"


2024-12-04 12:11:30,504 - root - INFO - 

******** Input Params ********
-->  country: de
-->  hotel_name: adlon-kempinski-berlin
-->  sort_by: most_relevant
-->  n_rows: -1
-->  stop_critera: None
************************


2024-12-04 12:11:30,504 - root - INFO - Creating URLs
2024-12-04 12:11:30,504 - root - INFO - Checking max offset parameter value
2024-12-04 12:11:30,845 - root - INFO - Offset parameter max value: 4050
2024-12-04 12:11:30,868 - root - INFO - Created URLs: 406
2024-12-04 12:11:30,869 - root - INFO - Progress Monitoring Thread Started
2024-12-04 12:11:30,869 - root - INFO - Starting Get Requests on 406 urls
2024-12-04 12:12:11,259 - root - INFO - Finished Get Requests in 40.4 seconds
2024-12-04 12:12:11,259 - root - INFO - Starting Parsing Responses: 406
2024-12-04 12:12:12,879 - root - INFO - Processed 16/406
2024-12-04 12:12:14,880 - root - INFO - Processed 114/406
2024-12-04 12:12:15,168 - root - INFO - Processes launched: 5
2024-12-04 12:12:16,880 - root - INFO 

Process Process-3:
Traceback (most recent call last):
  File "c:\Users\alan_\anaconda3\envs\Scraping\Lib\multiprocessing\process.py", line 314, in _bootstrap
    self.run()
  File "c:\Users\alan_\anaconda3\envs\Scraping\Lib\multiprocessing\process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "c:\GitHub Projects\Hotel_review_project\Scraping\core\scrape.py", line 393, in _parse_scraped_results
    in review_text_liked.lower()
       ^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'lower'


2024-12-04 12:25:36,041 - root - INFO - 

******** Input Params ********
-->  country: de
-->  hotel_name: clipper-city-home
-->  sort_by: most_relevant
-->  n_rows: -1
-->  stop_critera: None
************************


2024-12-04 12:25:36,041 - root - INFO - Creating URLs
2024-12-04 12:25:36,041 - root - INFO - Checking max offset parameter value
2024-12-04 12:25:36,363 - root - INFO - Offset parameter max value: 1550
2024-12-04 12:25:36,371 - root - INFO - Created URLs: 156
2024-12-04 12:25:36,372 - root - INFO - Progress Monitoring Thread Started
2024-12-04 12:25:36,372 - root - INFO - Starting Get Requests on 156 urls
2024-12-04 12:25:51,780 - root - INFO - Finished Get Requests in 15.4 seconds
2024-12-04 12:25:51,780 - root - INFO - Starting Parsing Responses: 156
2024-12-04 12:25:54,377 - root - INFO - Processed 59/156
2024-12-04 12:25:55,419 - root - INFO - Processes launched: 5
2024-12-04 12:25:56,378 - root - INFO - Processed 151/156
2024-12-04 12:25:56,614 - root - INFO - Fin

Process Process-4:
Traceback (most recent call last):
  File "c:\Users\alan_\anaconda3\envs\Scraping\Lib\multiprocessing\process.py", line 314, in _bootstrap
    self.run()
  File "c:\Users\alan_\anaconda3\envs\Scraping\Lib\multiprocessing\process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "c:\GitHub Projects\Hotel_review_project\Scraping\core\scrape.py", line 393, in _parse_scraped_results
    in review_text_liked.lower()
       ^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'lower'


2024-12-04 13:49:39,415 - root - INFO - 

******** Input Params ********
-->  country: de
-->  hotel_name: alexander-plaza
-->  sort_by: most_relevant
-->  n_rows: -1
-->  stop_critera: None
************************


2024-12-04 13:49:39,415 - root - INFO - Creating URLs
2024-12-04 13:49:39,415 - root - INFO - Checking max offset parameter value
2024-12-04 13:49:39,725 - root - INFO - Offset parameter max value: 3650
2024-12-04 13:49:39,749 - root - INFO - Created URLs: 366
2024-12-04 13:49:39,750 - root - INFO - Progress Monitoring Thread Started
2024-12-04 13:49:39,750 - root - INFO - Starting Get Requests on 366 urls
2024-12-04 13:50:16,186 - root - INFO - Finished Get Requests in 36.4 seconds
2024-12-04 13:50:16,186 - root - INFO - Starting Parsing Responses: 366
2024-12-04 13:50:17,759 - root - INFO - Processed 14/366
2024-12-04 13:50:19,760 - root - INFO - Processed 96/366
2024-12-04 13:50:20,434 - root - INFO - Processes launched: 5
2024-12-04 13:50:21,761 - root - INFO - Proces

Process Process-3:
Traceback (most recent call last):
  File "c:\Users\alan_\anaconda3\envs\Scraping\Lib\multiprocessing\process.py", line 314, in _bootstrap
    self.run()
  File "c:\Users\alan_\anaconda3\envs\Scraping\Lib\multiprocessing\process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "c:\GitHub Projects\Hotel_review_project\Scraping\core\scrape.py", line 393, in _parse_scraped_results
    in review_text_liked.lower()
       ^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'lower'


2024-12-04 15:33:16,745 - root - INFO - 

******** Input Params ********
-->  country: de
-->  hotel_name: sana-berlin
-->  sort_by: most_relevant
-->  n_rows: -1
-->  stop_critera: None
************************


2024-12-04 15:33:16,745 - root - INFO - Creating URLs
2024-12-04 15:33:16,745 - root - INFO - Checking max offset parameter value
2024-12-04 15:33:17,078 - root - INFO - Offset parameter max value: 5710
2024-12-04 15:33:17,111 - root - INFO - Created URLs: 572
2024-12-04 15:33:17,111 - root - INFO - Progress Monitoring Thread Started
2024-12-04 15:33:17,112 - root - INFO - Starting Get Requests on 572 urls
2024-12-04 15:34:14,508 - root - INFO - Finished Get Requests in 57.4 seconds
2024-12-04 15:34:14,508 - root - INFO - Starting Parsing Responses: 572
2024-12-04 15:34:17,126 - root - INFO - Processed 51/572
2024-12-04 15:34:18,522 - root - INFO - Processes launched: 5
2024-12-04 15:34:19,127 - root - INFO - Processed 189/572
2024-12-04 15:34:21,127 - root - INFO - Processed